In [97]:
## Langchain 1
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [98]:
LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [99]:
# Langsmith tracking and tracing
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")



In [100]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(
    model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001D98CF919A0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D98CF91C70> root_client=<openai.OpenAI object at 0x000001D98BCA87D0> root_async_client=<openai.AsyncOpenAI object at 0x000001D98CF91A90> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [101]:
# have paid version to use gpt-4o
# Example usage of the LLM
result = llm.invoke("What is the capital of France?")
print(result)

RuntimeError: Tracing using LangChainTracerV1 is no longer supported. Please set the LANGCHAIN_TRACING_V2 environment variable to enable tracing instead.

In [102]:
# free models 
from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")
print(model)

client=<groq.resources.chat.completions.Completions object at 0x000001D98CF93980> async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D98CF93E30> model_name='gemma2-9b-it' model_kwargs={} groq_api_key=SecretStr('**********')


In [ ]:
model.invoke("What is the capital of France?")  # Example usage of the Groq 

AIMessage(content='The capital of France is **Paris**. 🇫🇷  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_time': 0.027272727, 'prompt_time': 0.001856109, 'queue_time': 0.242995081, 'total_time': 0.029128836}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--838ef662-e9d5-4165-bf9e-a30d32d3d19e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31})

In [ ]:
### Prompt Engineering

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}"),
])

prompt

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [ ]:
### Chaining LLMs
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D98B91EE00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D98BC447C0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
response = chain.invoke({"question": "Explain CNN in computer vision."})
print(response)

content='Let\'s break down CNNs (Convolutional Neural Networks) in computer vision!\n\n**What are CNNs?**\n\nImagine you\'re trying to teach a computer to recognize a cat in a picture. You wouldn\'t just show it the whole image at once – you\'d likely point out features like pointy ears, whiskers, and a fluffy tail. CNNs work in a similar way.\n\nThey are a specialized type of artificial neural network designed specifically for processing visual information (images and videos). \n\n**Key Features of CNNs:**\n\n* **Convolutional Layers:** These are the heart of CNNs. They use filters (also called kernels) to scan the image, detecting specific features at different scales. Think of these filters as looking for edges, corners, textures, or even simple shapes like circles or squares.\n\n* **Pooling Layers:** After convolution, pooling layers reduce the size of the feature maps, making the network more efficient and robust to slight variations in the image. Imagine shrinking the image while

In [ ]:
## Output Parser
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
# Example usage of the output parser
chain = prompt | model | output_parser
response = chain.invoke({"question": "Explain CNN in computer vision."})
print(response)

Imagine you're trying to teach a computer to recognize cats in pictures. You could try showing it thousands of cat pictures and hoping it figures out the patterns on its own, but that's not very efficient. 

That's where Convolutional Neural Networks (CNNs) come in. They're like specialized brains designed specifically for understanding images.

Here's a simplified explanation:

**1. Convolutional Layers:** These are the heart of CNNs. They use filters (also called kernels) that scan the image, looking for specific patterns. Think of these filters like magnifying glasses that highlight edges, corners, or textures.

**2. Pooling Layers:**  After convolution, these layers summarize the information from the previous layer. They essentially shrink the image while retaining the most important features. Imagine a pool party where you only remember the general vibe and not every individual person.

**3. Fully Connected Layers:** These layers are similar to those in traditional neural networks

In [ ]:
### JSON Output Parser
from langchain_core.output_parsers import JsonOutputParser
json_output_parser = JsonOutputParser()
json_output_parser.get_format_instructions()


'Return a JSON object.'

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
json_output_parser = JsonOutputParser()

prompt=PromptTemplate(
    template="Answer the query \n {format_instructions}\n {question}",
    input_variables=["question"],
    partial_variables={"format_instructions": json_output_parser.get_format_instructions()},
)


In [ ]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='Answer the query \n {format_instructions}\n {question}')

In [ ]:
chain=prompt | model | json_output_parser
response =chain.invoke({"question": "tell me about transformmers in NLP."});print(response)

{'name': 'Transformers', 'description': 'A type of deep learning architecture specifically designed for natural language processing (NLP) tasks.', 'key_features': ['**Attention Mechanism:** Transformers leverage the attention mechanism to weigh the importance of different words in a sentence, allowing them to capture long-range dependencies.', '**Encoder-Decoder Structure:** Most transformers consist of an encoder that processes the input text and a decoder that generates the output.', '**Parallel Processing:** Unlike recurrent neural networks, transformers can process entire sequences of words in parallel, significantly speeding up training.', '**Self-Attention:**  Transformers use self-attention to allow each word in a sentence to attend to all other words, enabling a deeper understanding of context.', '**Positional Encodings:** Since transformers lack inherent sequential processing, they use positional encodings to provide information about the order of words.', '**Multi-Head Attent

In [ ]:
### OutputParser
from langchain_core.output_parsers import XMLOutputParser
output_parser=XMLOutputParser()
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question"),
        ("user","{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer.<response><answer>Your answer here</answer></response>.Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [ ]:
### OutputParser
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate

output_parser=XMLOutputParser()

prompt=PromptTemplate(
    template="Answer the user query \n {format_instruction}\n {query}\n ",
    input_variables=["query"],
    partial_variables={"format_instruction":output_parser.get_format_instructions()},
)
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'}, template='Answer the user query \n {format_instruction}\n {query}\n ')

In [ ]:
chain=prompt|model
response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

content='```xml\n<response>\n  <name>Langsmith</name>\n  <description>Langsmith is an open-source platform for developing and deploying large language models (LLMs). It provides a modular and extensible framework for building, training, and fine-tuning LLMs, as well as tools for deploying and managing them in production.</description>\n  <features>\n    <feature>Modular and extensible design</feature>\n    <feature>Support for multiple LLM architectures</feature>\n    <feature>Tools for data preprocessing and augmentation</feature>\n    <feature>Fine-tuning and evaluation tools</feature>\n    <feature>Deployment and management tools</feature>\n  </features>\n</response>\n```\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 195, 'total_tokens': 353, 'completion_time': 0.287272727, 'prompt_time': 0.008892394, 'queue_time': 0.24605681399999998, 'total_time': 0.296165121}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97

In [ ]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)


content='<response><answer>LangChain is a framework for developing applications powered by large language models (LLMs). It provides tools and components for tasks such as:\n\n- **Chain creation:** Combining multiple LLMs and other tools into workflows.\n- **Memory management:** Allowing LLMs to remember past interactions in a conversation.\n- **Prompt engineering:** Crafting effective prompts to elicit desired responses from LLMs.\n- **Data integration:** Connecting LLMs to external data sources.\n\nLangChain aims to simplify the development of LLM-based applications by providing a structured and modular approach.</answer></response>\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 39, 'total_tokens': 164, 'completion_time': 0.227272727, 'prompt_time': 0.002286529, 'queue_time': 0.243048301, 'total_time': 0.229559256}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id=

In [ ]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")



# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!'}

In [ ]:
### Without Pydantic
joke_query = "Tell me a joke ."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': "Why don't scientists trust atoms? Because they make up everything!"}

In [ ]:

from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate


actor_query = "Generate the shortened filmography for Tom Hanks."

output = model.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)

print(output.content)

Here is a shortened filmography for Tom Hanks, with movies enclosed in <movie></movie> tags:

* <movie>Splash</movie>
* <movie>Big</movie>
* <movie>Philadelphia</movie>
* <movie>Forrest Gump</movie>
* <movie>Saving Private Ryan</movie>
* <movie>Toy Story</movie>
* <movie>Cast Away</movie>
* <movie>The Da Vinci Code</movie>
* <movie>Apollo 13</movie>
* <movie>Cloud Atlas</movie>


This list highlights some of his most iconic and acclaimed roles. 




In [ ]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it")

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')